In [ ]:
# create database pydb;
# use pydb;
# CREATE TABLE book ( id INT NOT NULL AUTO_INCREMENT, title VARCHAR(200), pub VARCHAR(45), pages INT, author VARCHAR(100), PRIMARY KEY (id) );
#
# SELECT * FROM book;

In [11]:
# pip install pymysql
import pymysql

## 변수 선언 부분 ##
con, cur = None, None
data1, data2, data3, data4 = "", "", "", ""
row = None

## 메인 코드 부분 ##
con = pymysql.connect(host='127.0.0.1', user='webuser',
                      password='webuser', database='pydb', charset='utf8')
cur = con.cursor()

cur.execute("SELECT * FROM book")

print("ID    title    author    pages     ")
print("--------------------------------------------------------")

while (True):
    row = cur.fetchone()
    if row == None:
        break
    data1 = row[0]
    data2 = row[1]
    data3 = row[2]
    data4 = row[3]

    print("%5s   %15s   %15s  %10s\n" % (data1, data2, data3, data4))
    # print(f" ID: {data1}, title: {data2}, writer: {data3}")

con.close()

ID    title    author    pages     
--------------------------------------------------------
    1             샘플 제목             신우출판사             300 



In [12]:
import pymysql
from tkinter import *
from tkinter import messagebox

# 📌 MariaDB 접속 정보
DB_HOST = "127.0.0.1"
DB_USER = "webuser"
DB_PASSWORD = "webuser"
DB_NAME = "pydb"

# 📌 데이터 추가 함수
def insertData():
    try:
        con = pymysql.connect(
            host=DB_HOST, user=DB_USER, password=DB_PASSWORD,
            database=DB_NAME, charset="utf8"
        )
        cur = con.cursor()

        # 입력값 가져오기
        title = edtTitle.get().strip()
        pub = edtPub.get().strip()
        pages = edtPages.get().strip()
        author = edtAuthor.get().strip()

        if not title or not pub or not pages or not author:
            messagebox.showwarning("경고", "모든 값을 입력하세요!")
            return

        try:
            pages = int(pages)  # 정수 변환
        except ValueError:
            messagebox.showerror("오류", "페이지 수는 숫자로 입력하세요!")
            return

        # SQL 실행
        sql = "INSERT INTO book (title, pub, pages, author) VALUES (%s, %s, %s, %s)"
        cur.execute(sql, (title, pub, pages, author))

        con.commit()
        messagebox.showinfo("성공", "데이터 입력 성공")
        selectData()  # 입력 후 자동 조회

    except pymysql.MySQLError as e:
        messagebox.showerror("오류", f"DB 오류 발생: {e}")

    finally:
        if con:
            con.close()

# 📌 데이터 조회 함수
def selectData():
    try:
        con = pymysql.connect(
            host=DB_HOST, user=DB_USER, password=DB_PASSWORD,
            database=DB_NAME, charset="utf8"
        )
        cur = con.cursor()
        cur.execute("SELECT * FROM book")

        # 기존 데이터 삭제
        #         Listbox.delete(start, end)
        # 리스트박스(Listbox)에서 특정 범위의 항목을 삭제하는 메서드
        # 0, END를 사용하면 모든 항목을 삭제
        listData1.delete(0, END)
        listData2.delete(0, END)
        listData3.delete(0, END)
        listData4.delete(0, END)
        listData5.delete(0, END)

        # 컬럼명 추가
        #  Listbox.insert(index, value)
        # 리스트박스의 특정 위치에 항목을 추가
        # END를 사용하면 리스트의 마지막에 추가

        listData1.insert(END, "ID")
        listData2.insert(END, "제목")
        listData3.insert(END, "출판사")
        listData4.insert(END, "페이지 수")
        listData5.insert(END, "저자")
        listData1.insert(END, "-----------")
        listData2.insert(END, "-----------")
        listData3.insert(END, "-----------")
        listData4.insert(END, "-----------")
        listData5.insert(END, "-----------")

        # 데이터 추가
        for row in cur.fetchall():
            listData1.insert(END, row[0])  # ID
            listData2.insert(END, row[1])  # 제목
            listData3.insert(END, row[2])  # 출판사
            listData4.insert(END, row[3])  # 페이지 수
            listData5.insert(END, row[4])  # 저자

    except pymysql.MySQLError as e:
        messagebox.showerror("오류", f"DB 조회 오류 발생: {e}")

    finally:
        if con:
            con.close()

# 📌 데이터 수정 함수
def updateData():
    try:
        con = pymysql.connect(
            host=DB_HOST, user=DB_USER, password=DB_PASSWORD,
            database=DB_NAME, charset="utf8"
        )
        cur = con.cursor()

        # 입력값 가져오기
        book_id = edtId.get().strip()
        title = edtTitle.get().strip()
        pub = edtPub.get().strip()
        pages = edtPages.get().strip()
        author = edtAuthor.get().strip()

        if not book_id:
            messagebox.showwarning("경고", "ID를 입력하세요!")
            return

        try:
            book_id = int(book_id)  # 정수 변환
            pages = int(pages) if pages else None  # 페이지 수 변환
        except ValueError:
            messagebox.showerror("오류", "ID와 페이지 수는 숫자로 입력하세요!")
            return

        # SQL 실행
        sql = "UPDATE book SET title=%s, pub=%s, pages=%s, author=%s WHERE id=%s"
        cur.execute(sql, (title, pub, pages, author, book_id))

        if cur.rowcount == 0:
            messagebox.showwarning("경고", "ID를 찾을 수 없습니다.")
        else:
            con.commit()
            messagebox.showinfo("성공", "데이터 수정 성공")
            selectData()  # 수정 후 자동 조회

    except pymysql.MySQLError as e:
        messagebox.showerror("오류", f"DB 수정 오류 발생: {e}")

    finally:
        if con:
            con.close()

# 📌 데이터 삭제 함수
def deleteData():
    try:
        con = pymysql.connect(
            host=DB_HOST, user=DB_USER, password=DB_PASSWORD,
            database=DB_NAME, charset="utf8"
        )
        cur = con.cursor()

        book_id = edtId.get().strip()
        if not book_id:
            messagebox.showwarning("경고", "ID를 입력하세요!")
            return

        try:
            book_id = int(book_id)
        except ValueError:
            messagebox.showerror("오류", "ID는 숫자로 입력하세요!")
            return

        # SQL 실행
        sql = "DELETE FROM book WHERE id=%s"
        cur.execute(sql, (book_id,))

        if cur.rowcount == 0:
            messagebox.showwarning("경고", "ID를 찾을 수 없습니다.")
        else:
            con.commit()
            messagebox.showinfo("성공", "데이터 삭제 성공")
            selectData()  # 삭제 후 자동 조회

    except pymysql.MySQLError as e:
        messagebox.showerror("오류", f"DB 삭제 오류 발생: {e}")

    finally:
        if con:
            con.close()
# 📌 데이터 검색 함수
def searchData():
    try:
        con = pymysql.connect(
            host=DB_HOST, user=DB_USER, password=DB_PASSWORD,
            database=DB_NAME, charset="utf8"
        )
        cur = con.cursor()

        title = edtSearchTitle.get().strip()
        author = edtSearchAuthor.get().strip()

        if not title and not author:
            messagebox.showwarning("경고", "검색할 제목 또는 저자를 입력하세요!")
            return

        # SQL 실행 (동적 쿼리)
        sql = "SELECT * FROM book WHERE"
        params = []

        if title:
            sql += " title LIKE %s"
            params.append(f"%{title}%")

        if author:
            if title:
                sql += " AND"
            sql += " author LIKE %s"
            params.append(f"%{author}%")

        cur.execute(sql, tuple(params))

        # 기존 데이터 삭제
        listData1.delete(0, END)
        listData2.delete(0, END)
        listData3.delete(0, END)
        listData4.delete(0, END)
        listData5.delete(0, END)

        # 컬럼명 추가
        listData1.insert(END, "ID")
        listData2.insert(END, "제목")
        listData3.insert(END, "출판사")
        listData4.insert(END, "페이지 수")
        listData5.insert(END, "저자")
        listData1.insert(END, "-----------")
        listData2.insert(END, "-----------")
        listData3.insert(END, "-----------")
        listData4.insert(END, "-----------")
        listData5.insert(END, "-----------")

        # 검색 결과 추가
        results = cur.fetchall()
        if not results:
            messagebox.showinfo("알림", "검색 결과가 없습니다.")
        else:
            for row in results:
                listData1.insert(END, row[0])  # ID
                listData2.insert(END, row[1])  # 제목
                listData3.insert(END, row[2])  # 출판사
                listData4.insert(END, row[3])  # 페이지 수
                listData5.insert(END, row[4])  # 저자

    except pymysql.MySQLError as e:
        messagebox.showerror("오류", f"DB 검색 오류 발생: {e}")

    finally:
        if con:
            con.close()

# ✅ GUI 구성
window = Tk()
window.geometry("700x400")
window.title("도서 정보 관리")

# 📌 검색 필드
searchFrame = Frame(window)
searchFrame.pack(pady=10)

Label(searchFrame, text="제목 검색").pack(side=LEFT, padx=5)
edtSearchTitle = Entry(searchFrame, width=20)
edtSearchTitle.pack(side=LEFT, padx=5)

Label(searchFrame, text="저자 검색").pack(side=LEFT, padx=5)
edtSearchAuthor = Entry(searchFrame, width=20)
edtSearchAuthor.pack(side=LEFT, padx=5)

btnSearch = Button(searchFrame, text="검색", command=searchData)
btnSearch.pack(side=LEFT, padx=10)

# 📌 입력 필드
Label(window, text="ID").pack()
edtId = Entry(window, width=10)
edtId.pack()

Label(window, text="제목").pack()
edtTitle = Entry(window, width=20)
edtTitle.pack()

Label(window, text="출판사").pack()
edtPub = Entry(window, width=20)
edtPub.pack()

Label(window, text="페이지").pack()
edtPages = Entry(window, width=10)
edtPages.pack()

Label(window, text="저자").pack()
edtAuthor = Entry(window, width=20)
edtAuthor.pack()

# 📌 버튼
Button(window, text="입력", command=insertData).pack()
Button(window, text="조회", command=selectData).pack()
Button(window, text="수정", command=updateData).pack()
Button(window, text="삭제", command=deleteData).pack()

# 📌 리스트 박스
listFrame = Frame(window)
listFrame.pack()
listData1 = Listbox(listFrame, width=5)
listData1.pack(side=LEFT)
listData2 = Listbox(listFrame, width=20)
listData2.pack(side=LEFT)
listData3 = Listbox(listFrame, width=15)
listData3.pack(side=LEFT)
listData4 = Listbox(listFrame, width=10)
listData4.pack(side=LEFT)
listData5 = Listbox(listFrame, width=20)
listData5.pack(side=LEFT)

window.mainloop()
